In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
image=np.array([1,2,3,4,5,6,7,8,9],dtype=np.float32).reshape(1,3,3,1)

In [ ]:
plt.imshow(image.reshape(3,3), cmap='Greys')

In [ ]:
weight=tf.constant([[[[1.]],[[1.]]],
                    [[[1.]],[[1.]]]])
print(weight.shape)

In [ ]:
sess=tf.InteractiveSession() # eval할려면 InteractiveSession() 해야함
conv2d=tf.nn.conv2d(image,weight,strides=[1,1,1,1], padding='VALID') # 1, 좌우1, 위아래1, 1 // VALID 
conv2dImg=conv2d.eval() # 합성곱 연산

In [ ]:
conv2dImg.shape
conv2dImg

In [ ]:
for i, img in enumerate(conv2dImg):
    print(img.reshape(2,2))
    plt.subplot(1,2,i+1)
    plt.imshow(img.reshape(2,2), cmap='gray')

In [ ]:
conv2d=tf.nn.conv2d(image,weight,strides=[1,1,1,1], padding='SAME')
conv2dImg=conv2d.eval()

In [ ]:
conv2dImg.shape

In [ ]:
for i, img in enumerate(conv2dImg):
    print(img.reshape(3,3))
    plt.subplot(1,2,i+1)
    plt.imshow(img.reshape(3,3), cmap='gray')

In [2]:
# mnist 데이터 -> cnn 모델
import random
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist=input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
# hyper parameter
lr=0.01
epochs=15
batch_size=100

In [5]:
x=tf.placeholder(tf.float32,[None,28*28])
xImg=tf.reshape(x, [-1,28,28,1]) # None==-1
y=tf.placeholder(tf.float32, [None,10])

In [6]:
# tf.reset_default_graph()
# 입력 이미지 shape:(?, 28, 28, 1)
# 히든1 weight(w1=필터)
keep_prob=tf.placeholder(tf.float32)
w1=tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
# 3,3:필터의 높이와 너비, 1:채널의 개수, 32:필터의 개수
#conv->(?,28,28,32)
#relu->(?,28,28,32)
#pool->(?,14,14,32)
L1=tf.nn.conv2d(xImg,w1,strides=[1,1,1,1], padding="SAME")
L1=tf.nn.relu(L1)
L1=tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L1=tf.nn.dropout(L1,keep_prob=keep_prob)


w2=tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01)) # 위에 필터의 개수가 채널의 개수로, 필터의 개수는 원하는 만큼
#conv->(?,14,14,64)
#relu->(?,14,14,64)
#pool->(?,7,7,64)
#fflat ->(?,3136)
L2=tf.nn.conv2d(L1, w2, strides=[1,1,1,1], padding="SAME")
L2=tf.nn.relu(L2)
L2=tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
L2=tf.nn.dropout(L2,keep_prob=keep_prob)
L2_flat=tf.reshape(L2, [-1,7*7*64])


# 마지막 계층 FC 7*7*64 -> 10 output
w3=tf.get_variable('w3', shape=[7*7*64,10], initializer=tf.contrib.layers.xavier_initializer())
b=tf.Variable(tf.random_normal([10]))

hf=tf.matmul(L2_flat,w3)+b

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hf, labels=y))
opt=tf.train.AdamOptimizer(lr).minimize(cost)

sess=tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(epochs):
    avgCost=0
    totalBatch=int(mnist.train.num_examples/batch_size)
    for i in range(totalBatch):
        batchX,batchY=mnist.train.next_batch(batch_size)
        fd={x:batchX,y:batchY, keep_prob:0.7}
        cv, _=sess.run([cost,opt],feed_dict=fd)
        avgCost+=cv/totalBatch
    print('에폭:','%04d' %(epoch+1),'cost={:.5f}'.format(avgCost))
pred=tf.equal(tf.argmax(hf,1),tf.argmax(y,1))
acc=tf.reduce_mean(tf.cast(pred, tf.float32))
print('accuracy:', sess.run(acc, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1}))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

에폭: 0001 cost= 0.18303
에폭: 0002 cost= 0.09319
에폭: 0003 cost= 0.08426
에폭: 0004 cost= 0.07383
에폭: 0005 cost= 0.07516
에폭: 0006 cost= 0.07276
에폭: 0007 cost= 0.07092
에폭: 0008 cost= 0.06739
에폭: 0009 cost= 0.07400
에폭: 0010 cost= 0.06485
에폭: 0011 cost= 0.06754
에폭: 0012 cost= 0.06409
에폭: 0013 cost= 0.06550
에폭: 0014 cost= 0.06438
에폭: 0015 cost= 0.06583
accuracy: 0.9868
